In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time

In [2]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [25]:
class MyDriver:
    def __init__(self):
        options = webdriver.ChromeOptions()
        #options.add_argument("headless")
        #options.add_argument("disable-gpu")
        #options.add_argument("disable-infobars")
        #options.add_argument("no-sandbox")
        #options.add_argument("disable-dev-shm-usage")
        #options.add_argument("enable-experimental-web-platform-features")
        #options.add_argument("default-background-color FFFFFF00")
        
        self.__driver = webdriver.Chrome(options=options)
        self.__driver.get('https://klas.kw.ac.kr/')
        
    def accessToLogin(self, _id, _pw):
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div[2]/form/div[2]/button")))
        time.sleep(1)
        elemId = self.__driver.find_element_by_id("loginId")
        elemId.send_keys(_id)

        elemPW = self.__driver.find_element_by_id("loginPwd")
        elemPW.send_keys(_pw)
        elemPW.send_keys(Keys.ENTER)

        time.sleep(1)
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/header/div[1]/div/div[1]/button")))
        print("로그인 완료...")
        
    def accessToAssignmentPage(self):
        ## 과제 제출 페이지까지 가는 경로
        self.__driver.find_element_by_css_selector('div.toplogo > button').click()
        self.__driver.find_element_by_xpath('//*[@id="navbarHeader"]/div/div/div[2]/ul/li[1]/ul/li[6]/a').click()
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='appSelectSubj']/div[1]/div/div[2]/select")))
        time.sleep(0.5)
        # 수강학기 일단 변경
        self.__driver.find_element_by_css_selector('#appSelectSubj > div.col-md-5 > div > div.col-9 > select > option:nth-child(2)').click()
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='appSelectSubj']/div[1]/div/div[2]/select")))
        time.sleep(0.1)

        req = self.__driver.page_source

        self.__driver.get('https://klas.kw.ac.kr/')
        print("접근완료")
        return req
    
    def quitDriver(self):
        self.__driver.quit()
    
    def closeDriver(self):
        self.__driver.close()

In [42]:
myDriver = MyDriver()
myDriver.accessToLogin('2018203092', 'tkddlf^^12')
myDriver1 = MyDriver()
myDriver1.accessToLogin('2018203039', 'sdh9606^^')
time.sleep(2)
myDriver.quitDriver()
myDriver1.quitDriver()

로그인 완료...
로그인 완료...


In [21]:
import threading

def sum(low, high):
    total=0
    for i in range(low, high):
        total +=i
    print("Subjthread", total)
t = threading.Thread(target=sum, args=(1, 1000000))

t.start()

print("Main Thread")

Main Thread
Subjthread 499999500000


In [22]:
import threading, requests, time

def getHtml(url):
    resp = requests.get(url)
    time.sleep(1)
    print(url, len(resp.text), ' chars')

t1 = threading.Thread(target=getHtml, args=('http://google.com', ))
t1.start()

print("### End ###")

### End ###
http://google.com 14097  chars


In [30]:
data = []

class HtmlGetter(threading.Thread):
    def __init__(self, url):
        threading.Thread.__init__(self)
        self.url = url
        
    def run(self):
        resp = requests.get(self.url)
        time.sleep(1)
        print(self.url, len(resp.text), ' chars')
        global data
        data.append(self.url)
        print(data)
        
HtmlGetter('http://google.com').start()
HtmlGetter('http://google.com').start()

print("### End ###")

### End ###
http://google.com 14036  chars
['http://google.com']
http://google.com 14051  chars
['http://google.com', 'http://google.com']


# Thread 사용한 Current Page

****!주의해야할점****

- Crawling 함수는 어떻게 처리해야 할까?  


- 페이지 이동은, 과목별로 해야함을 명심하자!  
    - 따라서 클래스 쓰레드를 실행시킬때, 인자로 (id, pw, 과목명 혹은 과목ID)를 받도록 하자  
    
    - 페이지 이동 순서 : 로그인 -> 과목 목차에서 해당 과목 진입 -> 오른쪽 상단 테이블에서 각 카테코리 별 진입
    
    
- ThreadTimer 를 사용하여 주기적으로 실행 가능하게 되었다.  


- Thread의 데이터 동기화를 사용하여 DB에 저장할 수 있다.  

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

In [60]:
import threading
class MyThreadDriver(threading.Thread):
    def __init__(self):
        threading.Thread.__init__(self)
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        #options.add_argument("disable-gpu")
        #options.add_argument("disable-infobars")
        options.add_argument("no-sandbox")
        options.add_argument("disable-dev-shm-usage")
        #options.add_argument("enable-experimental-web-platform-features")
        #options.add_argument("default-background-color FFFFFF00")
        self.id = ''
        self.pw = ''
        self.subject_name = ''
        self.delay = 3
        self.__driver = webdriver.Chrome(options=options)
    
    def setCrawlingInfo(self, _id, _pw, _subject_name):
        self.id = _id
        self.pw = _pw
        self.subject_name = _subject_name
        
    def printLog(self, string):
        #print(string)
        pass
    
    def run(self):
        try:
            self.__driver.get('https://klas.kw.ac.kr/')
            self.accessToLogin()
            self.getIntoSubject()
            # 온라인 강의 크롤링
            self.goToAssignmentPage()
            # 과제 크롤링
            self.goToNoticePage()
            # 공지사항 크롤링
            self.goToAttachmentPage()
            # 첨부자료 크롤링
        except Exception as inst:
            print(inst)
            print(inst.args())
        finally:
            self.closeDriver()
        
    def accessToLogin(self):
        WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.ID, "loginId")))
        #time.sleep(1)
        elemId = self.__driver.find_element_by_id("loginId")
        elemId.send_keys(self.id)

        elemPW = self.__driver.find_element_by_id("loginPwd")
        elemPW.send_keys(self.pw)
        elemPW.send_keys(Keys.ENTER)

        #time.sleep(1)
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/header/div[1]/div/div[1]/button")))
        self.printLog("로그인 완료...")
        
    def getIntoSubject(self):
        # 해당 과목 페이지로 진입
        WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.CLASS_NAME, "subjectlist")))
        subjects = self.__driver.find_elements_by_css_selector(".subjectlist > li > div.left")
        for subject in subjects:
            name = subject.text.split()[0]
            if name == self.subject_name:
                subject.click()
                WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.CLASS_NAME, "subjectpresentbox")))
                self.printLog("과목 페이지 접근 완료")
                break
    
    # ---- 과제, 공지사항, 등등 페이지로 넘어갈 때에는 크롤링 하기전에만 Wait을 해주면 괜찮다. -----#
    # 과제 경로 -> 2, 2
    def goToAssignmentPage(self):
        ## 과제 제출 페이지까지 가는 경로
        #WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.CLASS_NAME, "subjectlist")))
        url = self.getCssSelector(2, 2)
        self.__driver.find_element_by_css_selector(url).click()
        self.printLog("과제 페이지 접근완료")
        time.sleep(1)
        self.goToPrevUrl()
        #time.sleep(1)
        
    # 공지사항
    def goToNoticePage(self):
        #WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.CLASS_NAME, "notice-list")))
        url = ".notice-list > div.bodtitle > a"
        self.__driver.find_element_by_css_selector(url).click()
        self.printLog("공지사항 페이지 접근완료")
        time.sleep(1)
        self.goToPrevUrl()
        #time.sleep(1)
        
    def goToAttachmentPage(self):
        #WebDriverWait(self.__driver, self.delay).until(EC.presence_of_element_located((By.CLASS_NAME, "subjectlist")))
        url = self.getCssSelector(2, 4)
        self.__driver.find_element_by_css_selector(url).click()
        self.printLog("강의자료 페이지 접근완료")
        time.sleep(1)
        self.goToPrevUrl()
        #time.sleep(1)
    
    def getCssSelector(self, div, li):
        return ".subjectpresentbox > div.tablelistbox > div > div:nth-child({}) > ul > li:nth-child({}) > a".format(div, li)
    
    def goToPrevUrl(self):
        self.__driver.execute_script("window.history.go(-1)")
        
    def quitDriver(self):
        self.__driver.quit()
    
    def closeDriver(self):
        print(threading.currentThread().getName() + " 종료")
        self.__driver.close()
        
        

In [61]:
# MyThreadDriver('2018203092', 'tkddlf^^12', "데이터베이스").start()
# MyThreadDriver('2018203039', 'sdh9606^^', "소프트웨어공학").start()
# MyThreadDriver('2018203092', 'tkddlf^^12', "정보시스템응용").start()

infoList = [ 
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"],
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"],
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"],
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"],
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"],
    ['2018203092', 'tkddlf^^12', "데이터베이스"], 
    ['2018203039', 'sdh9606^^', "소프트웨어공학"],
    ['2018203092', 'tkddlf^^12', "정보시스템응용"]
]


for data in infoList:
    myThreadDriver = MyThreadDriver()
    myThreadDriver.setCrawlingInfo(data[0], data[1], data[2])
    myThreadDriver.start()


#MyThreadDriver('2018203092', 'tkddlf^^12', "").start()
# MyThreadDriver('2018203039', 'sdh9606^^').start()
# MyThreadDriver('2018203092', 'tkddlf^^12').start()
# MyThreadDriver('2018203092', 'tkddlf^^12').start()
# MyThreadDriver('2018203039', 'sdh9606^^').start()
# MyThreadDriver('2018203092', 'tkddlf^^12').start()


Thread-137 종료
Thread-138 종료
Thread-139 종료


Exception in thread Thread-141:
Traceback (most recent call last):
  File "<ipython-input-60-18e1d8923800>", line 34, in run
  File "<ipython-input-60-18e1d8923800>", line 78, in goToAssignmentPage
  File "C:\Users\mosan\anaconda3\envs\KlasBot\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 598, in find_element_by_css_selector
    return self.find_element(by=By.CSS_SELECTOR, value=css_selector)
  File "C:\Users\mosan\anaconda3\envs\KlasBot\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\mosan\anaconda3\envs\KlasBot\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\mosan\anaconda3\envs\KlasBot\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementEx

Message: no such element: Unable to locate element: {"method":"css selector","selector":".subjectpresentbox > div.tablelistbox > div > div:nth-child(2) > ul > li:nth-child(2) > a"}
  (Session info: headless chrome=89.0.4389.82)

Thread-141 종료
Thread-140 종료
Thread-142 종료
Thread-143 종료
Thread-144 종료
Thread-145 종료
Thread-147 종료
Thread-146 종료
Thread-149 종료
Thread-148 종료
Thread-150 종료
Thread-151 종료
Thread-152 종료
Thread-153 종료
Thread-154 종료
